In [1]:
# import statements for rest of notebook
import matplotlib as mpl
import matplotlib.pyplot as plt
#plt.rc('text', usetex=True)
plt.rc('font', family='serif')
mpl.rcParams.update({'font.size': 20})
mpl.rcParams['figure.figsize'] = 8,8
import math as m   # importing math library outside function definition saves time 
import numpy as np

from IPython.display import Image
import scipy as sci
from scipy.misc import derivative
import scipy.optimize as opt
import sympy as sym

In [2]:
class Ray:
    """
        Ray Class:
        This is a container for multiple representations of the ray.
        Inputs:
            x -> The x cordinate where the ray starts propogation.
            y -> The y cordinate where the ray starts propogation.
            angle -> the angle relative to the y axis of the starting ray. 
        
        Stuff we can look up with the python dot notation eg: testRay.angle
            We have accesss to the inputs with:
                sourceLocation -> returns a tuple with the x and y input coords
                angle -> the propogation angle relative to the y axis. 
            There are multiple representations of the ray:
                N -> This is a sympy cordinate system for the vectors in sympy it is alligned with our y down x to the right cordinate system
                unitVector -> This is a vector with length one representing the direction of the ray.
                unitVectorXComponent -> X component of the unit vector.
                unitVectorYComponent -> Y component of the unit vector.
                slope -> The numerical value of the slope with respect to the cordinate system N.
                intercept -> The newmerical value of the y intercept acording to sourceLocation.
                paramaterization -> A sympy equation for the ray with t as the parameter for the line. 
    """
    def __init__(self, x, y, angle, wavelength=0):
        
        self.wavelength = wavelength
        self.sourceLocation = (x,y) # place where the ray originates.
        self.locationHistory = [self.sourceLocation] # keep track of the prevous interface points
        self.angle = angle # angle relative to the +y axis of the ray. 
        self.angleHistory = [self.angle]
        self.unitVector = [np.cos(self.angle), np.sin(self.angle)]
        self.slope = (self.unitVector[1]/self.unitVector[0])
        self.slopeFromRight = 0
        if(self.slope < 0):
            self.slopeFromRight = 1
        self._update_parameters()
        
    def _update_parameters(self):
        # This is how we define vectors in sympy.
        #self.N = smp.ReferenceFrame("N") # we need this "reference frame" to use the sympy vector stuff but its really just an angle reference.
        self.unitVector = [np.cos(self.angle),np.sin(self.angle)] # The "N.x" is the x hat in reference frame N. 
        self.slope = (self.unitVector[1] / self.unitVector[0]) # This is a silly numerical value (the evalf() condenses to numbers)
        self.yIntercept = self.sourceLocation[1] - self.sourceLocation[0]*self.slope # y = mx + b --> b = y - mx
        self.xIntercept = - self.yIntercept/self.slope # x = -b/m
        
    def update_location_history(self, x, y, angle):
        newPoint = (x,y)
        self.locationHistory.append(newPoint)
        self.sourceLocation = (x,y)
        self.angle = angle
        self.angleHistory.append(self.angle)
        self._update_parameters()
        
    def plot_ray(self):
        a = zip(*self.locationHistory)
        xPoints = a[0]
        yPoints = a[1]
        plt.plot(xPoints, yPoints, color='blue')
        
    def printString(self):
        return "{0}\n{1}\n{2}\n".format(self.slope, self.xIntercept,np.asarray(self.angleHistory)*180/np.pi)

    

In [3]:
#assuming list of rays are [x,y] pairs
def normalVector(function, value): # Value is the x value where the given ray intersects the lens surface
    fprime = derivative(function, value, dx=1e-13, order=21)
    return [-fprime/(1+fprime**2)**(0.5),1/(1+fprime**2)**(0.5)]#[x,y]
    
#expected inputs for line: [slope, x intercept], surface is a function, value is a reasonable estimate for where the zero occurs
def rayIntersectSurface(line, surface, value): 
    return opt.fsolve(lambda x: surface(x)-x*line[0] + line[0]*line[1], value, xtol=1e-10, maxfev=200)#f(x) - m(x-x0) = 0

def thetaCritical(n1, n2):
    return np.arcsin(n2/n1)

def rayFunc(listOfRays, surface1, n1, n2):#work on having our ray intersecting our surface
    for i in listOfRays:
        pointsOfIntersection = rayIntersectSurface([i.slope, i.xIntercept], surface1, i.sourceLocation[0])
        normalVectorPos = normalVector(surface1, pointsOfIntersection[0])
        theta1 = np.arccos(np.dot(i.unitVector, normalVectorPos))
        if(theta1 >= thetaCritical(n1(i.wavelength),n2(i.wavelength))):#internal reflection case
            continue
        theta2 = np.arcsin((n1(i.wavelength)/n2(i.wavelength))*np.sin(theta1))
        refractedRay1 = [-normalVectorPos[0]*np.cos(-theta2) + normalVectorPos[1]*np.sin(-theta2), \
                 -normalVectorPos[0]*np.sin(-theta2) - normalVectorPos[1]*np.cos(-theta2)]
        refractedRay2 = [-normalVectorPos[0]*np.cos(theta2) + normalVectorPos[1]*np.sin(theta2), \
                 -normalVectorPos[0]*np.sin(theta2) - normalVectorPos[1]*np.cos(theta2)]
        if(np.dot(refractedRay1, i.unitVector) < np.dot(refractedRay2, i.unitVector)):
            refractedRay = refractedRay1
        else:
            refractedRay = refractedRay2
        i.angleHistory.append(theta1)
        i.angleHistory.append(theta2)
        refractedXaxisTheta = np.arctan2(refractedRay[1],refractedRay[0])#new theta of refracted ray relative to x axis
        if(refractedXaxisTheta < 0):#gotta stay in the "right" quadrant
            refractedXaxisTheta += np.pi
        refractedRayIntercept = pointsOfIntersection[0] - refractedRay[0]/refractedRay[1]*surface1(pointsOfIntersection[0])#new x intercept
        i.update_location_history(pointsOfIntersection[0],surface1(pointsOfIntersection[0]),refractedXaxisTheta)
    return listOfRays


In [4]:
def point_source(x,y,numberOfRays, startAngle = 0, stopAngle = 2*np.pi):
    theRayList = []
    if startAngle == 0 and stopAngle == 2*np.pi:
        theAngleList = np.linspace(startAngle, stopAngle, numberOfRays+1) # this avoids two rays at 0 angle and keeps number of rays honest
    else:
        theAngleList = np.linspace(startAngle, stopAngle, numberOfRays)
    #print(theAngleList)
    for i, angle in enumerate(theAngleList):
        theRayList.append(Ray(x,y,angle, 0))#300+i*(400/numberOfRays)*10**-9))#color code
    return(theRayList)

def beam_source(x0, y0, x1, y1, numberOfRays):
    if (x1-x0) != 0 or (y1-y0) != 0:
        rayAngle = np.arctan2((x1-x0),-(y1-y0))
    else:
        raise(Exception("beam width must be non-zero"))
    xList = np.linspace(x0, x1, numberOfRays)
    yList = np.linspace(y0, y1, numberOfRays)
    pointList = list(zip(xList, yList))
    theRayList = []
    for i, point in enumerate(pointList):
        for j in range(100):
            ray = Ray(point[0],point[1],rayAngle, (400+j*(980/100))*10**-9)
            theRayList.append(ray) 
    return(theRayList)

#def converging_source()
def chromaticTester(x0, y0, angle, numberOfRays):
    return [Ray(x0,y0,angle,(400+i*(980/numberOfRays))*10**-9) for i in range(numberOfRays)]

In [16]:
def surface1(x):
    #set 1: biconvex lense
    #return -(-(x-5)**2 + 9)**(1/2) +5
    #return np.sin(2*x)+2
    #set 2: biconcave lense
    #return -(1.25-x/4)**2+5
    #return 0.000000001*x + 2.5
    return -(-(x-5)**2 + 9)**(1/2) +5
def surface2(x):
    #set 1: biconvex lense
    #return (-(x-5)**2 + 9)**(1/2) + 1
    #set 2: biconcave lense
    #return (x/4-1.25)**2+6
    #return 0.000000001*x + 6
    return (-(x-5)**2 + 9)**(1/2) +1


#this needs a better set up for encorperating the sellmeier relationship and the function form of the indices of refraction

B1 = 1.03961212
B2 = 0.231792344
B3 = 1.01046945

C1 = 6.00069867e-3
C2 = 2.00179144e-2
C3 = 1.0350653e2

def sellmeierEq(wavelength):
    #we expect our wavelength in nm but we convert to um
    if(wavelength == 0):
        return 1.5
    wavelength*=1e6
    return (1+2.00029547/(1-0.0121426017/wavelength**2)+0.298926886/(1-0.0538736236/wavelength**2)+1.80691843/(1-156.530829/wavelength**2))**.5
    #return (1+(B1*wavelength**2/(wavelength**2-C1)) + (B2*wavelength**2)/(wavelength**2-C2) +(B3*wavelength**2)/(wavelength**2-C3))**(1/2)

def nAir(wavelength):
    return 1
def nGlass(wavelength):
    return 1.55

#trialList= point_source(5,-1,10, np.pi/3, 2*np.pi/3)
trialList= beam_source(3.75,0,6.25,0,20)
#trialList = chromaticTester(3.5,0,np.pi/4,100)
a = rayFunc(trialList, surface1, nGlass, nAir)
b = rayFunc(a, surface2, nAir, nGlass)

t1 = np.linspace(-5,15,10000)
tt1 =  surface1(t1)
tt2 = surface2(t1)

C:\Anaconda2\envs\python3\lib\site-packages\ipykernel\__main__.py:11: RuntimeWarning: invalid value encountered in arcsin
C:\Anaconda2\envs\python3\lib\site-packages\ipykernel\__main__.py:8: RuntimeWarning: invalid value encountered in sqrt
C:\Anaconda2\envs\python3\lib\site-packages\ipykernel\__main__.py:15: RuntimeWarning: invalid value encountered in sqrt


In [17]:
fig = plt.figure()
sub = fig.add_subplot(111)
sub.plot(t1,tt1, 'r--')
sub.plot(t1,tt2, 'g--')
numRays = len(b)
for i in range(len(b)):
    xAx, yAx = zip(*b[i].locationHistory)
    xAxList = list(xAx)
    yAxList = list(yAx)
    length = len(xAx)
    if(b[i].slopeFromRight == 0):
        if(b[i].slope < 0):
            xAxList.append(xAxList[length-1]-5)
            yAxList.append(yAxList[length-1]-5*b[i].slope)
        else:
            xAxList.append(xAxList[length-1]+5)
            yAxList.append(yAxList[length-1]+5*b[i].slope)
    else:
        if(b[i].slope > 0):
            xAxList.append(xAxList[length-1]+5)
            yAxList.append(yAxList[length-1]+5*b[i].slope)
        else:
            xAxList.append(xAxList[length-1]-5)
            yAxList.append(yAxList[length-1]-5*b[i].slope)
        
    sub.plot(xAxList, yAxList, 'b', color=plt.cm.jet(i%100*1/(numRays/20)), alpha=0.08*(i//numRays/10+1))

plt.gca().invert_yaxis()
plt.ylim(0,10)
plt.xlim(3.5,6.5)
plt.show()

In [12]:
#for i in b:
#    print(i.angleHistory)